In [1]:
import numpy as np
import tensorflow as tf
import sys
sys.path.append('../')
from utils.datautils import loadfiledata
from utils.datautils import packnumericfeatures

In [2]:
TRAIN_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/train.csv"
TEST_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/eval.csv"

train_file_path = tf.keras.utils.get_file("train.csv", TRAIN_DATA_URL)
test_file_path = tf.keras.utils.get_file("eval.csv", TEST_DATA_URL)

In [3]:
LABEL_COLUMN = 'survived'
LABELS = [0, 1]
raw_train_data = loadfiledata.get_dataset(train_file_path, label_name=LABEL_COLUMN)
raw_test_data = loadfiledata.get_dataset(test_file_path, label_name=LABEL_COLUMN)

In [4]:
loadfiledata.show_batch(raw_train_data)

sex                 : [b'female' b'male' b'male' b'male' b'male']
age                 : [ 3. 39. 39. 28. 28.]
n_siblings_spouses  : [1 0 0 0 0]
parch               : [2 0 0 0 0]
fare                : [41.5792 24.15   13.     35.5     8.05  ]
class               : [b'Second' b'Third' b'Second' b'First' b'Third']
deck                : [b'unknown' b'unknown' b'unknown' b'A' b'unknown']
embark_town         : [b'Cherbourg' b'Southampton' b'Southampton' b'Southampton' b'Southampton']
alone               : [b'n' b'y' b'y' b'y' b'y']


In [5]:
CSV_COLUMNS = ['survived', 'sex', 'age', 'n_siblings_spouses', 'parch', 'fare', 'class', 'deck', 'embark_town', 'alone']
temp_dataset = loadfiledata.get_dataset(train_file_path, label_name=LABEL_COLUMN, column_names=CSV_COLUMNS)
loadfiledata.show_batch(temp_dataset)

sex                 : [b'male' b'female' b'female' b'female' b'male']
age                 : [28. 24. 28. 35. 28.]
n_siblings_spouses  : [0 1 1 1 1]
parch               : [0 0 0 0 0]
fare                : [ 8.05   26.     51.8625 83.475   7.75  ]
class               : [b'Third' b'Second' b'First' b'First' b'Third']
deck                : [b'unknown' b'unknown' b'D' b'C' b'unknown']
embark_town         : [b'Southampton' b'Southampton' b'Southampton' b'Southampton' b'Queenstown']
alone               : [b'y' b'n' b'n' b'n' b'n']


In [6]:
SELECT_COLUMNS = ['survived', 'age', 'n_siblings_spouses', 'class', 'deck', 'alone']
temp_dataset = loadfiledata.get_dataset(train_file_path, label_name=LABEL_COLUMN, select_columns=SELECT_COLUMNS)
loadfiledata.show_batch(temp_dataset)

age                 : [28. 26. 18. 28. 28.]
n_siblings_spouses  : [0 0 1 0 0]
class               : [b'First' b'Third' b'Third' b'Third' b'Third']
deck                : [b'unknown' b'unknown' b'unknown' b'unknown' b'unknown']
alone               : [b'y' b'y' b'n' b'y' b'y']


## Data preprocessing

`tf.feature_column` 可以做轉換

##### Continuous data

In [7]:
SELECT_COLUMNS = ['survived', 'age', 'n_siblings_spouses', 'parch', 'fare']
DEFAULTS = [0, 0.0, 0.0, 0.0, 0.0]
temp_dataset = loadfiledata.get_dataset(train_file_path, 
                           label_name='survived',
                           select_columns=SELECT_COLUMNS,
                           column_defaults = DEFAULTS)

loadfiledata.show_batch(temp_dataset)

age                 : [31. 23. 19. 43. 41.]
n_siblings_spouses  : [1. 0. 0. 1. 0.]
parch               : [0. 0. 0. 1. 1.]
fare                : [52.      7.55   10.1708 26.25   19.5   ]


In [8]:
# https://www.tensorflow.org/api_docs/python/tf/data/Iterator
example_batch, labels_batch = next(iter(temp_dataset))

In [9]:
packed_dataset = temp_dataset.map(loadfiledata.pack)
for features, labels in packed_dataset.take(1):
  print(features.numpy())
  print()
  print(labels.numpy())

[[28.      1.      0.     19.9667]
 [22.      0.      0.      7.2292]
 [22.      0.      0.      7.75  ]
 [27.      0.      0.     76.7292]
 [25.      0.      0.      0.    ]]

[0 0 1 1 1]


In [10]:
loadfiledata.show_batch(raw_train_data)

sex                 : [b'male' b'male' b'male' b'male' b'male']
age                 : [20. 57. 16. 22. 33.]
n_siblings_spouses  : [0 0 4 1 0]
parch               : [0 0 1 0 0]
fare                : [ 7.2292 12.35   39.6875  7.25    9.5   ]
class               : [b'Third' b'Second' b'Third' b'Third' b'Third']
deck                : [b'unknown' b'unknown' b'unknown' b'unknown' b'unknown']
embark_town         : [b'Cherbourg' b'Queenstown' b'Southampton' b'Southampton' b'Southampton']
alone               : [b'y' b'y' b'n' b'n' b'y']


In [11]:
example_batch, labels_batch = next(iter(temp_dataset))

In [12]:
NUMERIC_FEATURES = ['age','n_siblings_spouses','parch', 'fare']

packed_train_data = raw_train_data.map(
    packnumericfeatures.PackNumericFeatures(NUMERIC_FEATURES))

packed_test_data = raw_test_data.map(
    packnumericfeatures.PackNumericFeatures(NUMERIC_FEATURES))

In [13]:
loadfiledata.show_batch(packed_train_data)

sex                 : [b'male' b'male' b'female' b'male' b'male']
class               : [b'Third' b'Third' b'First' b'Second' b'First']
deck                : [b'unknown' b'unknown' b'C' b'unknown' b'unknown']
embark_town         : [b'Southampton' b'Southampton' b'Cherbourg' b'Southampton' b'Southampton']
alone               : [b'y' b'n' b'y' b'y' b'y']
numeric             : [[ 33.       0.       0.       9.5   ]
 [ 41.       2.       0.      14.1083]
 [ 36.       0.       0.     135.6333]
 [ 36.       0.       0.      13.    ]
 [ 56.       0.       0.      26.55  ]]


In [14]:
example_batch, labels_batch = next(iter(packed_train_data))

##### Data Normalization¶

In [15]:
import pandas as pd
# pandas module read_csv() function reads the CSV file into a DataFrame object.
desc = pd.read_csv(train_file_path)[NUMERIC_FEATURES].describe()
desc

,age,n_siblings_spouses,parch,fare
count,627.000000,627.000000,627.000000,627.000000
mean,29.631308,0.545455,0.379585,34.385399
std,12.511818,1.151090,0.792999,54.597730
min,0.750000,0.000000,0.000000,0.000000
25%,23.000000,0.000000,0.000000,7.895800
50%,28.000000,0.000000,0.000000,15.045800
75%,35.000000,1.000000,0.000000,31.387500
max,80.000000,8.000000,5.000000,512.329200


In [16]:

MEAN = np.array(desc.T['mean'])
STD = np.array(desc.T['std'])

In [17]:
import functools
normalizer = functools.partial(loadfiledata.normalize_numeric_data, mean=MEAN, std=STD)
# https://www.tensorflow.org/api_docs/python/tf/feature_column/numeric_column
# `tf.feature_column.numeric_column()` represents real valued or numerical features.
numeric_column = tf.feature_column.numeric_column('numeric', normalizer_fn=normalizer, shape=[len(NUMERIC_FEATURES)])
numeric_columns = [numeric_column]
numeric_column

NumericColumn(key='numeric', shape=(4,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function normalize_numeric_data at 0x000002707D2AA5E8>, mean=array([29.63130781,  0.54545455,  0.37958533, 34.38539856]), std=array([12.51181763,  1.1510896 ,  0.79299921, 54.5977305 ])))

In [18]:
example_batch['numeric']

<tf.Tensor: shape=(5, 4), dtype=float32, numpy=
array([[41.    ,  0.    ,  2.    , 20.2125],
       [ 2.    ,  3.    ,  1.    , 21.075 ],
       [28.    ,  1.    ,  0.    , 82.1708],
       [24.    ,  0.    ,  0.    ,  8.05  ],
       [45.    ,  0.    ,  0.    , 26.55  ]], dtype=float32)>

In [19]:
numeric_layer = tf.keras.layers.DenseFeatures(numeric_columns)
numeric_layer(example_batch).numpy()

array([[ 0.90863633, -0.47385937,  2.0434003 , -0.2595877 ],
       [-2.2084167 ,  2.1323671 ,  0.7823648 , -0.24379033],
       [-0.13038135,  0.39488277, -0.4786705 ,  0.8752269 ],
       [-0.45007908, -0.47385937, -0.4786705 , -0.4823534 ],
       [ 1.2283341 , -0.47385937, -0.4786705 , -0.14351146]],
      dtype=float32)

### Categorical data
使用 `tf.feature_column` API 為每個分類列創建一個帶有 `tf.feature_column.indicator_column` 的集合

In [20]:
CATEGORIES = {
    'sex': ['male', 'female'],
    'class' : ['First', 'Second', 'Third'],
    'deck' : ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'],
    'embark_town' : ['Cherbourg', 'Southhampton', 'Queenstown'],
    'alone' : ['y', 'n']
}

In [21]:
categorical_columns = []
for feature, vocab in CATEGORIES.items():
# Use the `tf.feature_column` API to create a collection with a `tf.feature_column.indicator_column` for each categorical column.
# https://www.tensorflow.org/api_docs/python/tf/feature_column/categorical_column_with_vocabulary_list
# https://www.tensorflow.org/api_docs/python/tf/feature_column/indicator_column
  cat_col = tf.feature_column.categorical_column_with_vocabulary_list(
        key=feature, vocabulary_list=vocab)
  categorical_columns.append(tf.feature_column.indicator_column(cat_col))

In [22]:
categorical_columns

[IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='sex', vocabulary_list=('male', 'female'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='class', vocabulary_list=('First', 'Second', 'Third'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='deck', vocabulary_list=('A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='embark_town', vocabulary_list=('Cherbourg', 'Southhampton', 'Queenstown'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='alone', vocabulary_list=('y', 'n'), dtype=tf.string, default_value=-1, num_oov_buckets=0))]

In [23]:
# `tf.keras.layers.DenseFeatures()` produces a dense Tensor based on given feature_columns.
categorical_layer = tf.keras.layers.DenseFeatures(categorical_columns)
print(categorical_layer(example_batch).numpy()[0])

[0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]


### Combined preprocessing layer

In [24]:
# 添加兩個特徵集合，並將它們傳遞給 tf.keras.layers.DenseFeatures，以創建一個輸入層，該層將提取並預處理這兩種輸入類型
preprocessing_layer = tf.keras.layers.DenseFeatures(categorical_columns+numeric_columns)

In [25]:
print(preprocessing_layer(example_batch).numpy()[0])

[ 0.          1.          0.          0.          1.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.90863633 -0.47385937  2.0434003  -0.2595877   0.          1.        ]
